In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
import matplotlib.dates as mdates
import statsmodels.api as sm
from scipy.stats import pearsonr
import itertools
from joblib import Parallel, delayed
import multiprocessing
import datetime

In [2]:
conn = wrds.Connection()

Enter your WRDS username [komalniraula]: kn2505
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  n


You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
years = list(range(1965, 2025)) # Bank secrecy act of 1970 required companies to follow specific reporting standards
df_list = []

for i, year in enumerate(years):
    print(f"Loading {year} ({i+1}/{len(years)})...")
    df_year = conn.raw_sql(f"""
        SELECT *
        FROM comp_na_daily_all.fundq
        WHERE indfmt = 'INDL'
          AND datafmt = 'STD'
          AND popsrc = 'D'
          AND consol = 'C'
          AND EXTRACT(YEAR FROM datadate) = {year}
    """, date_cols=['datadate'])
    df_list.append(df_year)

df_comp_main = pd.concat(df_list, ignore_index=True)

Loading 1965 (1/60)...
Loading 1966 (2/60)...
Loading 1967 (3/60)...
Loading 1968 (4/60)...
Loading 1969 (5/60)...
Loading 1970 (6/60)...
Loading 1971 (7/60)...
Loading 1972 (8/60)...
Loading 1973 (9/60)...
Loading 1974 (10/60)...
Loading 1975 (11/60)...
Loading 1976 (12/60)...
Loading 1977 (13/60)...
Loading 1978 (14/60)...
Loading 1979 (15/60)...
Loading 1980 (16/60)...
Loading 1981 (17/60)...
Loading 1982 (18/60)...
Loading 1983 (19/60)...
Loading 1984 (20/60)...
Loading 1985 (21/60)...
Loading 1986 (22/60)...
Loading 1987 (23/60)...
Loading 1988 (24/60)...
Loading 1989 (25/60)...
Loading 1990 (26/60)...
Loading 1991 (27/60)...
Loading 1992 (28/60)...
Loading 1993 (29/60)...
Loading 1994 (30/60)...
Loading 1995 (31/60)...
Loading 1996 (32/60)...
Loading 1997 (33/60)...
Loading 1998 (34/60)...
Loading 1999 (35/60)...
Loading 2000 (36/60)...
Loading 2001 (37/60)...
Loading 2002 (38/60)...
Loading 2003 (39/60)...
Loading 2004 (40/60)...
Loading 2005 (41/60)...
Loading 2006 (42/60)...
L

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4312/449054777.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comp_main = pd.concat(df_list, ignore_index=True)
/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4312/449054777.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comp_main = pd.concat(df_list, ignore_index=True)


In [4]:
df_comp = df_comp_main.copy()

In [5]:
df_comp_ = df_comp[['gvkey', 'rdq', 'fyearq', 'fqtr', 'cusip', 'fyr', 'prchq', 'prclq', 'cdvcy', 'dvy', 'dvpspq', 'optdrq', 
                    'datafqtr', 'cdvcy']]
df_comp_= df_comp_[df_comp_['gvkey'] == '160329']
df_comp_

,gvkey,rdq,fyearq,fqtr,cusip,fyr,prchq,prclq,cdvcy,dvy,dvpspq,optdrq,datafqtr,cdvcy
1024046,160329,<NA>,2002,1,02079K305,12,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2002Q1,<NA>
1024047,160329,<NA>,2002,2,02079K305,12,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2002Q2,<NA>
1024048,160329,<NA>,2002,3,02079K305,12,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2002Q3,<NA>
1024049,160329,<NA>,2002,4,02079K305,12,<NA>,<NA>,<NA>,0.0,<NA>,<NA>,2002Q4,<NA>
1067616,160329,<NA>,2003,1,02079K305,12,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2003Q1,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979483,160329,2024-01-30,2023,4,02079K305,12,142.68,120.2057,<NA>,0.0,0.0,<NA>,2023Q4,<NA>
2029884,160329,2024-04-25,2024,1,02079K305,12,153.78,130.665,<NA>,0.0,0.0,<NA>,2024Q1,<NA>
2029885,160329,2024-07-23,2024,2,02079K305,12,186.05,149.6,<NA>,2466.0,0.2,<NA>,2024Q2,<NA>
2029886,160329,2024-10-29,2024,3,02079K305,12,191.75,147.215,<NA>,4921.0,0.2,<NA>,2024Q3,<NA>


In [6]:
df_comp = df_comp.sort_values(by=['gvkey', 'datadate'])

# Calculate quarterly CapEx from year-to-date capxy
df_comp['capxq'] = df_comp.groupby('gvkey')['capxy'].diff()
df_comp['fqtr'] = df_comp['fqtr'].fillna(df_comp['datadate'].dt.quarter)
df_comp.loc[df_comp['fqtr'] == 1, 'capxq'] = df_comp['capxy']

# Core financial ratios
df_comp['roa'] = df_comp['niq'] / df_comp['atq']
df_comp['operating_margin'] = df_comp['oiadpq'] / df_comp['saleq']
df_comp['gross_margin'] = (df_comp['saleq'] - df_comp['cogsq']) / df_comp['saleq']
df_comp['revenue_growth'] = df_comp.groupby('gvkey')['saleq'].pct_change()
df_comp['r_and_d_intensity'] = df_comp['xrdq'] / df_comp['atq']
df_comp['capex_intensity'] = df_comp['capxq'] / df_comp['atq']

# Accruals (Sloan)
df_comp['atq_lag'] = df_comp.groupby('gvkey')['atq'].shift(1)
df_comp['d_actq'] = df_comp.groupby('gvkey')['actq'].diff()
df_comp['d_cheq'] = df_comp.groupby('gvkey')['cheq'].diff()
df_comp['d_lctq'] = df_comp.groupby('gvkey')['lctq'].diff()
df_comp['d_dd1q'] = df_comp.groupby('gvkey')['dd1q'].diff()
df_comp['d_txpq'] = df_comp.groupby('gvkey')['txpq'].diff()

df_comp['sloan_accruals'] = (
    (df_comp['d_actq'] - df_comp['d_cheq']) -
    (df_comp['d_lctq'] - df_comp['d_dd1q'] - df_comp['d_txpq']) -
    df_comp['dpq']
) / df_comp['atq_lag']

# Rolling 8-quarter volatility
df_comp['roa_stability'] = df_comp.groupby('gvkey')['roa'].rolling(8).std().reset_index(level=0, drop=True)
df_comp['revenue_growth_stability'] = df_comp.groupby('gvkey')['revenue_growth'].rolling(8).std().reset_index(level=0, drop=True)

# Winsorization function
def winsorize(series, lower=0.01, upper=0.99):
    return series.clip(lower=series.quantile(lower), upper=series.quantile(upper))

ratio_columns = [
    'roa', 'operating_margin', 'gross_margin', 'sloan_accruals',
    'revenue_growth', 'r_and_d_intensity', 'capex_intensity',
    'roa_stability', 'revenue_growth_stability'
]

for col in ratio_columns:
    df_comp[col] = winsorize(df_comp[col])

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4312/1156743967.py:12: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_comp['revenue_growth'] = df_comp.groupby('gvkey')['saleq'].pct_change()
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:4653: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:4653: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


In [7]:
# Keep the row with fqtr=1 when there are duplicate gvkey+datadate
df_comp = df_comp.sort_values(['gvkey', 'datadate', 'fqtr'])  # ascending: fqtr=1 will be last
df_comp = df_comp.drop_duplicates(subset=['gvkey', 'datadate'], keep='last')

In [8]:
df_comp['datadate'] = pd.to_datetime(df_comp['datadate'])
df_comp['year']=df_comp['datadate'].dt.year
df_comp.head()

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,atq_lag,d_actq,d_cheq,d_lctq,d_dd1q,d_txpq,sloan_accruals,roa_stability,revenue_growth_stability,year
3663,001000,1966-03-31,1966,1,12,INDL,C,D,STD,AE.2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,1966
3664,001000,1966-06-30,1966,2,12,INDL,C,D,STD,AE.2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,1966
3665,001000,1966-09-30,1966,3,12,INDL,C,D,STD,AE.2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,1966
3666,001000,1966-12-31,1966,4,12,INDL,C,D,STD,AE.2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,1966
10531,001000,1967-03-31,1967,1,12,INDL,C,D,STD,AE.2,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,1967


In [9]:
# Preferred Stock: PSTKRQ → PSTKNQ → 0
df_comp['ps'] = df_comp['pstkrq'].fillna(df_comp['pstknq']).fillna(0)

# Stockholders’ Equity (SEQ): SEQQ → ATQ - LTQ → CEQQ + PSTKQ + TXDITCQ
df_comp['seqq'] = df_comp['seqq'].fillna(df_comp['atq'] - df_comp['ltq'])
df_comp['seqq'] = df_comp['seqq'].fillna(df_comp['ceqq'] + df_comp['pstkq'] + df_comp['txditcq'])

# Book Equity = SEQ - Preferred Stock
df_comp['be'] = df_comp['seqq'] - df_comp['ps']
df_comp['be'] = df_comp['be'].where(df_comp['be'] > 0)

In [10]:
# sort each firm's accounting records in chronological order
df_comp = df_comp.sort_values(by=['gvkey', 'datadate'])

# add a counter to track how many quarter of data exist per firm
df_comp['count'] = df_comp.groupby('gvkey').cumcount()

# retain only relevant columns for merging and future use
df_comp = df_comp[['gvkey', 'datadate', 'year', 'be', 'count'] + ratio_columns]
df_comp

,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,r_and_d_intensity,capex_intensity,roa_stability,revenue_growth_stability
3663,001000,1966-03-31,1966,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
3664,001000,1966-06-30,1966,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
3665,001000,1966-09-30,1966,<NA>,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN
3666,001000,1966-12-31,1966,<NA>,3,<NA>,0.086806,0.300595,<NA>,<NA>,<NA>,<NA>,NaN,NaN
10531,001000,1967-03-31,1967,<NA>,4,<NA>,<NA>,<NA>,<NA>,0.0,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989336,356687,2023-12-31,2023,87.766,7,-0.060509,-inf,NaN,<NA>,NaN,<NA>,0.102836,0.819372,NaN
2037730,356687,2024-03-31,2024,150.626,8,-0.027518,-inf,NaN,0.029428,NaN,<NA>,0.039107,0.068628,NaN
2037731,356687,2024-06-30,2024,209.337,9,-0.008274,-inf,NaN,-0.009785,NaN,<NA>,0.042664,0.033313,NaN
2037732,356687,2024-09-30,2024,209.318,10,-0.004379,-inf,NaN,-0.002431,NaN,<NA>,0.03859,0.029810,NaN


In [11]:
df_comp.isnull().sum()

gvkey                             0
datadate                          0
year                              0
be                           615310
count                             0
roa                          511701
operating_margin             449508
gross_margin                 440382
sloan_accruals              1773020
revenue_growth               334172
r_and_d_intensity           1563741
capex_intensity              825229
roa_stability                831324
revenue_growth_stability     707659
dtype: int64

In [12]:
# CRSP Block      
df_crsp = conn.raw_sql("""
    SELECT a.permno, a.permco, a.date, b.shrcd, b.exchcd, b.ticker, b.cusip,
    a.ret, a.retx, a.shrout, a.prc
    FROM crsp.msf AS a
    LEFT JOIN crsp.msenames AS b
      ON a.permno = b.permno
      AND b.namedt <= a.date
      AND a.date <= b.nameendt
      WHERE a.date BETWEEN '01/01/1948' AND '12/31/2024'
      AND b.exchcd BETWEEN 1 AND 3  -- NYSE, AMEX, NASDAQ
      AND b.shrcd IN (10, 11)       -- Common stocks only
""", date_cols=['date'])
#df_crsp = pd.read_csv("crsp.csv")
df_crsp.head()

,permno,permco,date,shrcd,exchcd,ticker,cusip,ret,retx,shrout,prc
0,10000,7952,1986-01-31,10,3,OMFGA,68391610,<NA>,<NA>,3680.0,-4.375
1,10000,7952,1986-02-28,10,3,OMFGA,68391610,-0.257143,-0.257143,3680.0,-3.25
2,10000,7952,1986-03-31,10,3,OMFGA,68391610,0.365385,0.365385,3680.0,-4.4375
3,10000,7952,1986-04-30,10,3,OMFGA,68391610,-0.098592,-0.098592,3793.0,-4.0
4,10000,7952,1986-05-30,10,3,OMFGA,68391610,-0.222656,-0.222656,3793.0,-3.10938


In [13]:
# change variable format to int
df_crsp[['permco','permno','shrcd','exchcd']]=df_crsp[['permco','permno','shrcd','exchcd']].astype(int)

df_crsp['date'] = pd.to_datetime(df_crsp['date'])

# roll forward to the end of the month, if current date is not the end of the month
df_crsp['jdate']=df_crsp['date']+MonthEnd(0) 

In [14]:
# fill missing return values with 0
df_crsp['ret'] = df_crsp['ret'].fillna(0)   # total return (includes delisting, as provided by CRSP)
df_crsp['retx'] = df_crsp['retx'].fillna(0) # return excluding delisting

# calculate market equity (in $ thousands)
df_crsp['ME'] = df_crsp['prc'].abs() * df_crsp['shrout']  # prc is negative if bid quote, so take abs()

# drop columns no longer needed
df_crsp = df_crsp.drop(['prc', 'shrout'], axis=1)

# sort by date, firm ID (permco), and market equity
df_crsp = df_crsp.sort_values(by=['date', 'permco', 'ME'])

# view cleaned data
df_crsp.head()

,permno,permco,date,shrcd,exchcd,ticker,cusip,ret,retx,jdate,ME
94388,17670,74,1948-01-31,10,1,<NA>,02342410,-0.045064,-0.062232,1948-01-31,6828.125
155114,18702,267,1948-01-31,10,1,<NA>,03333010,-0.02381,-0.02381,1948-01-31,25922.25
262852,20714,584,1948-01-31,10,1,<NA>,09788010,-0.058824,-0.058824,1948-01-31,40512.0
167367,11287,921,1948-01-31,10,1,<NA>,17286410,0.054217,0.054217,1948-01-31,38915.625
286474,21151,994,1948-01-31,10,1,<NA>,23111110,0.007692,-0.007692,1948-01-31,14399.625


In [15]:
# sum of ME across all permnos belonging to the same permco on a given date
crsp_sumME = df_crsp.groupby(['jdate', 'permco'])['ME'].sum().reset_index()

# find the largest single ME (security) per permco/date
crsp_maxME = df_crsp.groupby(['jdate', 'permco'])['ME'].max().reset_index()

# keep only the permno with the largest ME per firm/date
df_crsp1 = pd.merge(df_crsp, crsp_maxME, how='inner', on=['jdate', 'permco', 'ME'])

# drop ME (security-level) and replace it with firm-level total ME
df_crsp1 = df_crsp1.drop(['ME'], axis=1)

# add back the total firm-level ME
df_crsp2 = pd.merge(df_crsp1, crsp_sumME, how='inner', on=['jdate', 'permco'])  # New 'ME' is total ME

# sort and drop duplicates (in case of multiple securities with identical ME)
df_crsp2 = df_crsp2.sort_values(by=['permno', 'jdate']).drop_duplicates()

df_crsp2.head()

,permno,permco,date,shrcd,exchcd,ticker,cusip,ret,retx,jdate,ME
1238014,10000,7952,1986-01-31,10,3,OMFGA,68391610,0.0,0.0,1986-01-31,16100.0
1243659,10000,7952,1986-02-28,10,3,OMFGA,68391610,-0.257143,-0.257143,1986-02-28,11960.0
1249305,10000,7952,1986-03-31,10,3,OMFGA,68391610,0.365385,0.365385,1986-03-31,16330.0
1254957,10000,7952,1986-04-30,10,3,OMFGA,68391610,-0.098592,-0.098592,1986-04-30,15172.0
1260614,10000,7952,1986-05-30,10,3,OMFGA,68391610,-0.222656,-0.222656,1986-05-31,11793.87834


In [16]:
# extract year and month from jdate
df_crsp2['year'] = df_crsp2['jdate'].dt.year
df_crsp2['month'] = df_crsp2['jdate'].dt.month

In [17]:
def assign_quarter(month):
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'

df_crsp2['qtr'] = df_crsp2['month'].apply(assign_quarter)

df_crsp2['qkey'] = df_crsp2['year'].astype(str) + '-' + df_crsp2['qtr']

df_crsp2['1+retx'] = 1 + df_crsp2['retx'].fillna(0)

df_crsp2['cumretx_q'] = df_crsp2.groupby(['permno', 'qkey'])['1+retx'].cumprod()
df_crsp2

,permno,permco,date,shrcd,exchcd,ticker,cusip,ret,retx,jdate,ME,year,month,qtr,qkey,1+retx,cumretx_q
1238014,10000,7952,1986-01-31,10,3,OMFGA,68391610,0.0,0.0,1986-01-31,16100.0,1986,1,Q1,1986-Q1,1.0,1.0
1243659,10000,7952,1986-02-28,10,3,OMFGA,68391610,-0.257143,-0.257143,1986-02-28,11960.0,1986,2,Q1,1986-Q1,0.742857,0.742857
1249305,10000,7952,1986-03-31,10,3,OMFGA,68391610,0.365385,0.365385,1986-03-31,16330.0,1986,3,Q1,1986-Q1,1.365385,1.014286
1254957,10000,7952,1986-04-30,10,3,OMFGA,68391610,-0.098592,-0.098592,1986-04-30,15172.0,1986,4,Q2,1986-Q2,0.901408,0.901408
1260614,10000,7952,1986-05-30,10,3,OMFGA,68391610,-0.222656,-0.222656,1986-05-31,11793.87834,1986,5,Q2,1986-Q2,0.777344,0.700704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3559840,93436,53453,2024-08-30,11,3,TSLA,88160R10,-0.077391,-0.077391,2024-08-31,684004370.4,2024,8,Q3,2024-Q3,0.922609,1.082018
3563671,93436,53453,2024-09-30,11,3,TSLA,88160R10,0.221942,0.221942,2024-09-30,839047410.0,2024,9,Q3,2024-Q3,1.221942,1.322164
3567481,93436,53453,2024-10-31,11,3,TSLA,88160R10,-0.045025,-0.045025,2024-10-31,802033523.1006,2024,10,Q4,2024-Q4,0.954975,0.954975
3571279,93436,53453,2024-11-29,11,3,TSLA,88160R10,0.381469,0.381469,2024-11-30,1107984309.6,2024,11,Q4,2024-Q4,1.381469,1.319268


In [18]:
# CCM Block
df_ccm = conn.raw_sql("""
    SELECT gvkey, lpermno AS permno, linktype, linkprim, 
           linkdt, linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE SUBSTR(linktype, 1, 1) = 'L'       -- Only active "link" types (e.g., LU, LC)
      AND (linkprim = 'C' OR linkprim = 'P') -- Primary links: 'C' (most reliable), 'P' (used if 'C' missing)
""", date_cols=['linkdt', 'linkenddt'])
#df_ccm = pd.read_csv("ccm.csv")
df_ccm.head()

,gvkey,permno,linktype,linkprim,linkdt,linkenddt
0,001000,25881.0,LU,P,1970-11-13,1978-06-30
1,001001,10015.0,LU,P,1983-09-20,1986-07-31
2,001002,10023.0,LC,C,1972-12-14,1973-06-05
3,001003,10031.0,LU,C,1983-12-07,1989-08-16
4,001004,54594.0,LU,P,1972-04-24,NaT


In [19]:
df_ccm['linkenddt'] = pd.to_datetime(df_ccm['linkenddt'])

# if linkenddt is missing then set to today date
df_ccm['linkenddt']=df_ccm['linkenddt'].fillna(pd.to_datetime('today')) # setting those to today's date makes sure the link is considered valid when merging historical Compustat and CRSP data

# associate each Compustat firm (gvkey) with its corresponding CRSP stock (permno) to eventually match with stock return data
df_ccm1=pd.merge(df_comp,
                 df_ccm,how='left',on=['gvkey']) # keeps all compustat records and attaches CRSP info if available

#Convert datadate to datetime if not already
df_ccm1['datadate'] = pd.to_datetime(df_ccm1['datadate'])

# Create jdate as the calendar month-end date
df_ccm1['jdate'] = df_ccm1['datadate'] + pd.offsets.MonthEnd(0)

# extract year and month from datadate
df_ccm1['year'] = df_ccm1['datadate'].dt.year
df_ccm1['month'] = df_ccm1['datadate'].dt.month
df_ccm1

,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,...,capex_intensity,roa_stability,revenue_growth_stability,permno,linktype,linkprim,linkdt,linkenddt,jdate,month
0,001000,1966-03-31,1966,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,NaN,NaN,25881.0,LU,P,1970-11-13,1978-06-30,1966-03-31,3
1,001000,1966-06-30,1966,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,NaN,NaN,25881.0,LU,P,1970-11-13,1978-06-30,1966-06-30,6
2,001000,1966-09-30,1966,<NA>,2,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,NaN,NaN,25881.0,LU,P,1970-11-13,1978-06-30,1966-09-30,9
3,001000,1966-12-31,1966,<NA>,3,<NA>,0.086806,0.300595,<NA>,<NA>,...,<NA>,NaN,NaN,25881.0,LU,P,1970-11-13,1978-06-30,1966-12-31,12
4,001000,1967-03-31,1967,<NA>,4,<NA>,<NA>,<NA>,<NA>,0.0,...,<NA>,NaN,NaN,25881.0,LU,P,1970-11-13,1978-06-30,1967-03-31,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414938,356687,2023-12-31,2023,87.766,7,-0.060509,-inf,NaN,<NA>,NaN,...,0.102836,0.819372,NaN,<NA>,<NA>,<NA>,NaT,NaT,2023-12-31,12
2414939,356687,2024-03-31,2024,150.626,8,-0.027518,-inf,NaN,0.029428,NaN,...,0.039107,0.068628,NaN,<NA>,<NA>,<NA>,NaT,NaT,2024-03-31,3
2414940,356687,2024-06-30,2024,209.337,9,-0.008274,-inf,NaN,-0.009785,NaN,...,0.042664,0.033313,NaN,<NA>,<NA>,<NA>,NaT,NaT,2024-06-30,6
2414941,356687,2024-09-30,2024,209.318,10,-0.004379,-inf,NaN,-0.002431,NaN,...,0.03859,0.029810,NaN,<NA>,<NA>,<NA>,NaT,NaT,2024-09-30,9


In [20]:
# only keep rows where jdate (portfolio assignment date) falls within the link window (linkdt to linkenddt)
df_ccm2 = df_ccm1[
    (df_ccm1['jdate'] >= df_ccm1['linkdt']) &
    (df_ccm1['jdate'] <= df_ccm1['linkenddt'])
]

In [21]:
invalid_months = df_ccm2[~df_ccm2['month'].isin([3, 6, 9, 12])]
print(f"Number of non-quarter-end rows: {len(invalid_months)}")

Number of non-quarter-end rows: 217754


In [22]:
def map_to_quarter_end(month):
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'  # 10, 11, 12 → 12

df_ccm2['quarter'] = df_ccm2['month'].apply(map_to_quarter_end)

duplicates = df_ccm2[df_ccm2.duplicated(subset=['permno', 'month', 'year'], keep=False)]
print(duplicates.sort_values(by=['permno', 'year', 'month']))

          gvkey   datadate  year     be  count       roa  operating_margin  \
592172   008175 1991-07-31  1991  1.987     40 -0.017041         -0.092823   
1383750  025188 1991-07-31  1991   <NA>      0      <NA>         -0.130435   
592173   008175 1991-10-31  1991  1.994     41  0.001782          0.398964   
1383751  025188 1991-10-31  1991   <NA>      1      <NA>          0.026761   
592174   008175 1992-01-31  1992  1.552     42  0.003002          0.057103   
...         ...        ...   ...    ...    ...       ...               ...   
214975   003626 1985-12-31  1985   <NA>      8      <NA>              <NA>   
364065   005425 1986-12-31  1986   <NA>     10 -0.641953         -7.388889   
1044186  014361 1986-12-31  1986   <NA>      0 -0.325497              <NA>   
364066   005425 1987-03-31  1987   <NA>     11 -1.458955         -1.292453   
1044187  014361 1987-03-31  1987   <NA>      1      <NA>         -1.635514   

         gross_margin  sloan_accruals  revenue_growth  ...  roa

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4312/3696967175.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ccm2['quarter'] = df_ccm2['month'].apply(map_to_quarter_end)


In [23]:
duplicates = df_ccm2[df_ccm2.duplicated(subset=['permno', 'month', 'year'], keep=False)]
print(duplicates.sort_values(by=['permno', 'year', 'month']))

          gvkey   datadate  year     be  count       roa  operating_margin  \
592172   008175 1991-07-31  1991  1.987     40 -0.017041         -0.092823   
1383750  025188 1991-07-31  1991   <NA>      0      <NA>         -0.130435   
592173   008175 1991-10-31  1991  1.994     41  0.001782          0.398964   
1383751  025188 1991-10-31  1991   <NA>      1      <NA>          0.026761   
592174   008175 1992-01-31  1992  1.552     42  0.003002          0.057103   
...         ...        ...   ...    ...    ...       ...               ...   
214975   003626 1985-12-31  1985   <NA>      8      <NA>              <NA>   
364065   005425 1986-12-31  1986   <NA>     10 -0.641953         -7.388889   
1044186  014361 1986-12-31  1986   <NA>      0 -0.325497              <NA>   
364066   005425 1987-03-31  1987   <NA>     11 -1.458955         -1.292453   
1044187  014361 1987-03-31  1987   <NA>      1      <NA>         -1.635514   

         gross_margin  sloan_accruals  revenue_growth  ...  roa

In [24]:
# Extract non-duplicate rows
non_duplicates = df_ccm2.drop_duplicates(subset=['permno', 'month', 'year'], keep=False)

# Shuffle duplicates so we can select randomly when needed
duplicates = duplicates.sample(frac=1, random_state=42)

# Apply logic to resolve duplicates
def resolve_duplicates(group):
    non_nan = group[group['be'].notna()]
    if len(non_nan) == 1:
        return non_nan
    else:
        return group.iloc[[0]]  # randomly keep one

deduped = duplicates.groupby(['permno', 'month', 'year'], group_keys=False).apply(resolve_duplicates)

# Combine back with the non-duplicates
df_ccm1_cleaned = pd.concat([non_duplicates, deduped], ignore_index=True)

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4312/1379524675.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  deduped = duplicates.groupby(['permno', 'month', 'year'], group_keys=False).apply(resolve_duplicates)


In [25]:
duplicates = df_ccm1_cleaned[df_ccm1_cleaned.duplicated(subset=['permno', 'month', 'year'], keep=False)]
print(duplicates.sort_values(by=['permno', 'year', 'month']))

Empty DataFrame
Columns: [gvkey, datadate, year, be, count, roa, operating_margin, gross_margin, sloan_accruals, revenue_growth, r_and_d_intensity, capex_intensity, roa_stability, revenue_growth_stability, permno, linktype, linkprim, linkdt, linkenddt, jdate, month, quarter]
Index: []

[0 rows x 22 columns]


In [26]:
df_merged = pd.merge(
    df_ccm1_cleaned,   # your final Compustat data with jdate
    df_crsp2,          # your CRSP data with jdate
    on=['permno', 'jdate', 'year', 'month'],
    how='left'         # keep all Compustat rows, match CRSP when available
)

# BE is in millions, ME is in thousands → multiply BE by 1000 to match units
df_merged['beme'] = df_merged['be'] * 1000 / df_merged['ME']
df_merged

,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,...,ticker,cusip,ret,retx,ME,qtr,qkey,1+retx,cumretx_q,beme
0,001000,1970-12-31,1970,10.544,19,<NA>,0.360656,-1.518546,<NA>,0.020658,...,AE,00003210,0.111111,0.111111,26550.0,Q4,1970-Q4,1.111111,1.111111,0.397137
1,001000,1971-03-31,1971,<NA>,20,<NA>,<NA>,<NA>,<NA>,-0.268889,...,AE,00003210,-0.123596,-0.123596,25886.25,Q1,1971-Q1,0.876404,0.974999,<NA>
2,001000,1971-06-30,1971,<NA>,21,<NA>,<NA>,<NA>,<NA>,0.238256,...,AE,00003210,-0.043478,-0.043478,21903.75,Q2,1971-Q2,0.956522,0.846154,<NA>
3,001000,1971-09-30,1971,<NA>,22,<NA>,<NA>,<NA>,<NA>,-0.193222,...,AE,00003210,-0.119048,-0.119048,12279.375,Q3,1971-Q3,0.880952,0.560606,<NA>
4,001000,1971-12-31,1971,8.381,23,-0.073099,0.020706,-2.077363,<NA>,-0.012915,...,AE,00003210,0.393939,0.393939,15266.25,Q4,1971-Q4,1.393939,1.243243,0.548989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1370196,003626,1985-06-30,1985,2.06,6,0.034755,0.143634,0.25854,<NA>,-0.570953,...,RPMM,22819010,-0.075472,-0.075472,3583.125,Q2,1985-Q2,0.924528,0.960784,0.574917
1370197,003626,1985-09-30,1985,2.181,7,0.030406,0.104902,0.244118,<NA>,-0.208075,...,RPMM,22819010,-0.01,-0.01,3619.6875,Q3,1985-Q3,0.99,1.010204,0.602538
1370198,001642,1985-12-31,1985,5.471,16,0.005084,0.023923,0.268968,<NA>,0.55804,...,RPMM,22819010,0.191919,0.191919,11025.625,Q4,1985-Q4,1.191919,1.191919,0.496208
1370199,005425,1987-03-31,1987,<NA>,11,-1.458955,-1.292453,0.801887,<NA>,1.944444,...,IMIT,44972510,0.465116,0.465116,7810.07092,Q1,1987-Q1,1.465116,1.340426,<NA>


In [27]:
df_merged.isnull().sum()

gvkey                             0
datadate                          0
year                              0
be                           319983
count                             0
roa                          315359
operating_margin             307447
gross_margin                 304071
sloan_accruals              1215187
revenue_growth               225036
r_and_d_intensity           1014944
capex_intensity              534338
roa_stability                485617
revenue_growth_stability     351477
permno                            0
linktype                          0
linkprim                          0
linkdt                            0
linkenddt                         0
jdate                             0
month                             0
quarter                           0
permco                       386775
date                         386775
shrcd                        386775
exchcd                       386775
ticker                       387574
cusip                       

In [28]:
print("Rows with missing ME:", df_merged['ME'].isna().sum())
print("Total rows:", len(df_merged))

Rows with missing ME: 386775
Total rows: 1370201


In [29]:
missing_permnos = df_merged[df_merged['ME'].isna()]['permno'].unique()
print(f"# permnos not matched: {len(missing_permnos)}")
print("Example permnos:", missing_permnos[:5])

# permnos not matched: 12725
Example permnos: <FloatingArray>
[10082.0, 66405.0, 47351.0, 28601.0, 65453.0]
Length: 5, dtype: Float64


In [30]:
# For df_ccm1_cleaned (Compustat side)
num_nan_permno_ccm = df_ccm1_cleaned['permno'].isna().sum()
print(f"NaN permno in df_ccm1_cleaned: {num_nan_permno_ccm}")

# For df_crsp2 (CRSP side)
num_nan_permco_crsp = df_crsp2['permco'].isna().sum()
print(f"NaN permcno in df_crsp2: {num_nan_permco_crsp}")

NaN permno in df_ccm1_cleaned: 0
NaN permcno in df_crsp2: 0


In [31]:
unmatched = df_merged[df_merged['permco'].isna()]
print(unmatched[['gvkey', 'permno', 'jdate']].drop_duplicates().head())

      gvkey   permno      jdate
487  001011  10082.0 1991-09-30
488  001011  10082.0 1991-12-31
489  001011  10082.0 1992-03-31
490  001011  10082.0 1992-06-30
491  001011  10082.0 1992-09-30


In [32]:
df_merged_clean = df_merged[df_merged['ME'].notna()]

In [33]:
df_merged_clean.isnull().sum()

gvkey                            0
datadate                         0
year                             0
be                           92447
count                            0
roa                          93219
operating_margin             87770
gross_margin                 85263
sloan_accruals              859191
revenue_growth               23401
r_and_d_intensity           697415
capex_intensity             270877
roa_stability               226725
revenue_growth_stability    127079
permno                           0
linktype                         0
linkprim                         0
linkdt                           0
linkenddt                        0
jdate                            0
month                            0
quarter                          0
permco                           0
date                             0
shrcd                            0
exchcd                           0
ticker                         799
cusip                            0
ret                 

In [34]:
df_merged_clean['ticker'] = df_merged_clean.groupby('permno')['ticker'].transform('first')

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4312/2246511775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_clean['ticker'] = df_merged_clean.groupby('permno')['ticker'].transform('first')


In [35]:
df_merged_clean = df_merged_clean[df_merged_clean['jdate'] > datetime.datetime(1974, 12, 31)]

df_merged_clean = df_merged_clean[df_merged_clean['beme'].notna()]

df_merged_clean

,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,...,ticker,cusip,ret,retx,ME,qtr,qkey,1+retx,cumretx_q,beme
17,001000,1975-03-31,1975,9.893,36,<NA>,<NA>,<NA>,<NA>,-0.00895,...,AE,00003210,0.142857,0.142857,6651.0,Q1,1975-Q1,1.142857,1.411765,1.487445
18,001000,1975-06-30,1975,10.768,37,<NA>,<NA>,<NA>,<NA>,0.27853,...,AE,00003210,0.146341,0.146341,6201.0625,Q2,1975-Q2,1.146341,0.979166,1.736477
19,001000,1975-09-30,1975,11.386,38,<NA>,<NA>,<NA>,<NA>,-0.091337,...,AE,00003210,0.230769,0.230769,8444.0,Q3,1975-Q3,1.230769,1.361701,1.348413
20,001000,1975-12-31,1975,10.24,39,0.02535,0.432544,-1.8569,<NA>,-0.060052,...,AE,00003210,0.029412,0.029412,9235.625,Q4,1975-Q4,1.029412,1.09375,1.10875
21,001000,1976-03-31,1976,10.993,40,0.022317,0.105941,0.313571,<NA>,0.315551,...,AE,00003210,0.08,0.08,14276.25,Q1,1976-Q1,1.08,1.542857,0.77002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1370193,061337,1997-12-31,1997,78.229,15,-0.024582,0.118764,0.507741,<NA>,0.125894,...,GSCN,67000B10,-0.342857,-0.342857,214348.5,Q4,1997-Q4,0.657143,0.495512,0.364962
1370195,014647,1986-08-31,1986,8.875,3,-0.737196,-1.473616,0.712803,<NA>,<NA>,...,XPLR,92333310,0.007143,-0.035714,8231.625,Q3,1986-Q3,0.964286,0.818182,1.078159
1370196,003626,1985-06-30,1985,2.06,6,0.034755,0.143634,0.25854,<NA>,-0.570953,...,CRAN,22819010,-0.075472,-0.075472,3583.125,Q2,1985-Q2,0.924528,0.960784,0.574917
1370197,003626,1985-09-30,1985,2.181,7,0.030406,0.104902,0.244118,<NA>,-0.208075,...,CRAN,22819010,-0.01,-0.01,3619.6875,Q3,1985-Q3,0.99,1.010204,0.602538


In [36]:
# Count number of unique quarters per permno
permno_quarter_counts = df_merged_clean.groupby('permno')['jdate'].nunique()

# Filter permnos with at least 8 unique quarters (i.e., 2 years)
valid_permnos = permno_quarter_counts[permno_quarter_counts >= 8].index

# Keep only those rows
df_merged_clean = df_merged_clean[df_merged_clean['permno'].isin(valid_permnos)].copy()
df_merged_clean

,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,...,ticker,cusip,ret,retx,ME,qtr,qkey,1+retx,cumretx_q,beme
17,001000,1975-03-31,1975,9.893,36,<NA>,<NA>,<NA>,<NA>,-0.00895,...,AE,00003210,0.142857,0.142857,6651.0,Q1,1975-Q1,1.142857,1.411765,1.487445
18,001000,1975-06-30,1975,10.768,37,<NA>,<NA>,<NA>,<NA>,0.27853,...,AE,00003210,0.146341,0.146341,6201.0625,Q2,1975-Q2,1.146341,0.979166,1.736477
19,001000,1975-09-30,1975,11.386,38,<NA>,<NA>,<NA>,<NA>,-0.091337,...,AE,00003210,0.230769,0.230769,8444.0,Q3,1975-Q3,1.230769,1.361701,1.348413
20,001000,1975-12-31,1975,10.24,39,0.02535,0.432544,-1.8569,<NA>,-0.060052,...,AE,00003210,0.029412,0.029412,9235.625,Q4,1975-Q4,1.029412,1.09375,1.10875
21,001000,1976-03-31,1976,10.993,40,0.022317,0.105941,0.313571,<NA>,0.315551,...,AE,00003210,0.08,0.08,14276.25,Q1,1976-Q1,1.08,1.542857,0.77002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1370193,061337,1997-12-31,1997,78.229,15,-0.024582,0.118764,0.507741,<NA>,0.125894,...,GSCN,67000B10,-0.342857,-0.342857,214348.5,Q4,1997-Q4,0.657143,0.495512,0.364962
1370195,014647,1986-08-31,1986,8.875,3,-0.737196,-1.473616,0.712803,<NA>,<NA>,...,XPLR,92333310,0.007143,-0.035714,8231.625,Q3,1986-Q3,0.964286,0.818182,1.078159
1370196,003626,1985-06-30,1985,2.06,6,0.034755,0.143634,0.25854,<NA>,-0.570953,...,CRAN,22819010,-0.075472,-0.075472,3583.125,Q2,1985-Q2,0.924528,0.960784,0.574917
1370197,003626,1985-09-30,1985,2.181,7,0.030406,0.104902,0.244118,<NA>,-0.208075,...,CRAN,22819010,-0.01,-0.01,3619.6875,Q3,1985-Q3,0.99,1.010204,0.602538


In [37]:
df_merged_clean.isnull().sum()

gvkey                            0
datadate                         0
year                             0
be                               0
count                            0
roa                           4855
operating_margin             19355
gross_margin                 13145
sloan_accruals              727855
revenue_growth                2584
r_and_d_intensity           571983
capex_intensity             166271
roa_stability               115381
revenue_growth_stability     78310
permno                           0
linktype                         0
linkprim                         0
linkdt                           0
linkenddt                        0
jdate                            0
month                            0
quarter                          0
permco                           0
date                             0
shrcd                            0
exchcd                           0
ticker                          45
cusip                            0
ret                 

In [40]:
def fill_missing_custom(df, features, id_col='permno', time_col='jdate'):
    df = df.copy()
    df = df.sort_values([id_col, time_col])

    for col in features:
        print(f"Filling missing for: {col}")

        # Step 1: Interpolate in-between missing values
        df[col] = df.groupby(id_col)[col].transform(
            lambda x: x.interpolate(method='linear', limit_direction='both')
        )

        # Step 2: Extrapolate forward using average market change
        # We'll store updated values here and assign all at once
        updates = []

        for firm_id, group in df.groupby(id_col):
            group = group.copy()
            for i in range(1, len(group)):
                if pd.isna(group.iloc[i][col]):
                    prev_val = group.iloc[i - 1][col]
                    prev_date = group.iloc[i - 1][time_col]
                    curr_date = group.iloc[i][time_col]

                    if pd.isna(prev_val):
                        continue

                    # Get others' average change from prev_date → curr_date
                    others_prev = df[(df[time_col] == prev_date) & (df[id_col] != firm_id)][col].dropna()
                    others_curr = df[(df[time_col] == curr_date) & (df[id_col] != firm_id)][col].dropna()

                    if not others_prev.empty and not others_curr.empty:
                        avg_pct_change = (others_curr.mean() - others_prev.mean()) / others_prev.mean()
                        new_val = prev_val * (1 + avg_pct_change)
                        updates.append((group.index[i], new_val))

        # Apply all updates for this column
        for idx, val in updates:
            df.at[idx, col] = val

    return df

knn_features = [
    'roa', 'operating_margin', 'gross_margin',
    'sloan_accruals', 'revenue_growth',
    'r_and_d_intensity', 'capex_intensity',
    'roa_stability', 'revenue_growth_stability'
]

df_filled = fill_missing_custom(df_merged_clean, knn_features)
df_filled

Filling missing for: roa
Filling missing for: operating_margin


/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4312/3400609669.py:33: RuntimeWarning: invalid value encountered in scalar subtract
  avg_pct_change = (others_curr.mean() - others_prev.mean()) / others_prev.mean()
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4312/3400609669.py:33: RuntimeWarning: invalid value encountered in scalar subtract
  avg_pct_change = (others_curr.mean()

Filling missing for: gross_margin


/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/var/folde

Filling missing for: sloan_accruals
Filling missing for: revenue_growth


/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/komalniraula/myenv/lib/python3.13/site-packages/numpy/_core/_methods.py:135: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/kom

Filling missing for: r_and_d_intensity
Filling missing for: capex_intensity
Filling missing for: roa_stability
Filling missing for: revenue_growth_stability


,gvkey,datadate,year,be,count,roa,operating_margin,gross_margin,sloan_accruals,revenue_growth,...,ticker,cusip,ret,retx,ME,qtr,qkey,1+retx,cumretx_q,beme
562255,012994,1986-03-31,1986,5.588,2,0.029621,0.108738,0.122232,<NA>,-0.062055,...,GFGC,36720410,0.0252,0.01,6217.8125,Q1,1986-Q1,1.01,1.030612,0.898708
562256,012994,1986-06-30,1986,5.432,3,-0.004248,-0.021113,0.010271,<NA>,-0.54081,...,GFGC,36720410,-0.013069,-0.029703,6033.125,Q2,1986-Q2,0.970297,0.970297,0.900363
562257,012994,1986-09-30,1986,5.18,4,-0.015723,-0.265903,-0.201146,<NA>,-0.50214,...,GFGC,36720410,-0.003077,-0.019231,6317.625,Q3,1986-Q3,0.980769,1.040816,0.819928
562258,012994,1986-12-31,1986,5.25,5,0.012606,0.070839,0.091576,<NA>,2.122636,...,GFGC,36720410,0.015,0.0,6937.0,Q4,1986-Q4,1.0,1.09804,0.756811
562259,012994,1987-03-31,1987,5.488,6,0.024801,0.11394,0.131481,<NA>,0.182235,...,GFGC,36720410,0.0368,0.02,6317.625,Q1,1987-Q1,1.02,0.910715,0.868681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330083,184996,2023-12-31,2023,62634.0,63,0.074359,0.082012,0.225295,-0.013658,0.077816,...,TSLA,88160R10,0.034988,0.034988,791408800.0,Q4,2023-Q4,1.034988,0.993046,0.079142
1330084,184996,2024-03-31,2024,64378.0,64,0.012726,0.054974,0.232008,0.013891,-0.153614,...,TSLA,88160R10,-0.129235,-0.129235,560588301.25034,Q1,2024-Q1,0.870765,0.707462,0.11484
1330085,184996,2024-06-30,2024,66468.0,65,0.012408,0.087333,0.229647,-0.01116,0.197127,...,TSLA,88160R10,0.111186,0.111186,632155363.2,Q2,2024-Q2,1.111186,1.125662,0.105145
1330086,184996,2024-09-30,2024,69931.0,66,0.018131,0.110079,0.251966,-0.031817,-0.012471,...,TSLA,88160R10,0.221942,0.221942,839047410.0,Q3,2024-Q3,1.221942,1.322164,0.083346


In [41]:
df_filled.isnull().sum()

gvkey                            0
datadate                         0
year                             0
be                               0
count                            0
roa                             38
operating_margin               807
gross_margin                  1711
sloan_accruals              543269
revenue_growth                2105
r_and_d_intensity           410064
capex_intensity              46646
roa_stability                 1545
revenue_growth_stability     11777
permno                           0
linktype                         0
linkprim                         0
linkdt                           0
linkenddt                        0
jdate                            0
month                            0
quarter                          0
permco                           0
date                             0
shrcd                            0
exchcd                           0
ticker                          45
cusip                            0
ret                 

In [42]:
df_final = df_filled[['gvkey', 'permno', 'ticker', 'cumretx_q', 'beme', 'roa', 'operating_margin', 'gross_margin', 
                      'revenue_growth', 'capex_intensity', 
                      'roa_stability', 'revenue_growth_stability', 'year', 'quarter', 'jdate']]

In [43]:
# Rank BEME within each quarter (ascending for growth)
df_final['beme_pct_rank'] = df_final.groupby('quarter')['beme'].rank(pct=True, method='first')

# Filter bottom 30% (growth stocks)
df_growth_portfolio = df_final[df_final['beme_pct_rank'] <= 0.3].copy()

df_growth_portfolio

/var/folders/m4/8tn_t7fn3n999rq0tn3djx1w0000gn/T/ipykernel_4312/1364522422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['beme_pct_rank'] = df_final.groupby('quarter')['beme'].rank(pct=True, method='first')


,gvkey,permno,ticker,cumretx_q,beme,roa,operating_margin,gross_margin,revenue_growth,capex_intensity,roa_stability,revenue_growth_stability,year,quarter,jdate,beme_pct_rank
530922,012165,10008.0,GACO,1.213676,0.365155,0.019269,0.139914,0.339251,0.771964,0.015725,0.019758,0.441593,1986,Q1,1986-03-31,0.294805
548590,012622,10010.0,CBOT,1.0,0.284875,0.012412,0.103623,0.592131,-0.13042,0.009602,0.009422,0.351080,1986,Q1,1986-01-31,0.212232
548596,012622,10010.0,CBOT,1.166667,0.359478,0.023699,0.110345,0.518897,0.069006,0.025768,0.009422,0.190848,1987,Q3,1987-07-31,0.281509
548608,012622,10010.0,CBOT,1.25,0.123272,0.024862,0.142737,0.322984,0.152647,0.036632,0.003487,0.127214,1990,Q3,1990-07-31,0.060701
548609,012622,10010.0,CBOT,0.852941,0.230516,0.028277,0.1745,0.111473,0.111141,-0.012986,0.003485,0.121452,1990,Q4,1990-10-31,0.155287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330083,184996,93436.0,TSLA,0.993046,0.079142,0.074359,0.082012,0.225295,0.077816,0.021638,0.016941,0.118269,2023,Q4,2023-12-31,0.031368
1330084,184996,93436.0,TSLA,0.707462,0.11484,0.012726,0.054974,0.232008,-0.153614,0.025424,0.018973,0.138303,2024,Q1,2024-03-31,0.056483
1330085,184996,93436.0,TSLA,1.125662,0.105145,0.012408,0.087333,0.229647,0.197127,0.020136,0.020744,0.140593,2024,Q2,2024-06-30,0.049627
1330086,184996,93436.0,TSLA,1.322164,0.083346,0.018131,0.110079,0.251966,-0.012471,0.029311,0.020845,0.114321,2024,Q3,2024-09-30,0.033368


In [44]:
# 📌 Value Portfolio (Top 30% by BEME → High BEME ratios)
df_value_portfolio = df_final[df_final['beme_pct_rank'] >= 0.7].copy()

df_value_portfolio

,gvkey,permno,ticker,cumretx_q,beme,roa,operating_margin,gross_margin,revenue_growth,capex_intensity,roa_stability,revenue_growth_stability,year,quarter,jdate,beme_pct_rank
562255,012994,10001.0,GFGC,1.030612,0.898708,0.029621,0.108738,0.122232,-0.062055,0.022155,0.018870,1.303014,1986,Q1,1986-03-31,0.713374
562256,012994,10001.0,GFGC,0.970297,0.900363,-0.004248,-0.021113,0.010271,-0.54081,0.022155,0.018870,1.303014,1986,Q2,1986-06-30,0.722596
562260,012994,10001.0,GFGC,0.921568,0.922172,-0.001189,-0.01072,0.029481,-0.536635,0.022155,0.018870,1.303014,1987,Q2,1987-06-30,0.733438
562263,012994,10001.0,GFGC,1.042553,0.915405,0.043095,0.143474,0.159632,0.316523,0.022155,0.022002,1.155961,1988,Q1,1988-03-31,0.721958
562264,012994,10001.0,GFGC,1.020408,0.889032,0.004778,0.036627,0.070965,-0.634907,0.022155,0.021628,1.166721,1988,Q2,1988-06-30,0.7166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1324471,184259,93434.0,SANW,2.273116,1.584895,-0.022654,-0.73508,0.370888,-0.388976,0.000777,0.106148,0.588939,2024,Q4,2024-12-31,0.904525
1181137,144356,93435.0,SCEI,0.210526,3.861884,0.11108,0.188165,0.386019,-0.318641,0.01076,0.362367,0.335520,2011,Q2,2011-06-30,0.99071
1181138,144356,93435.0,SCEI,0.725,5.499844,0.027294,0.213003,0.377376,-0.254899,0.0,0.143858,0.364647,2011,Q3,2011-09-30,0.994773
1181139,144356,93435.0,SCEI,1.149426,5.000126,0.032554,0.193857,0.234102,0.645889,-0.001564,0.112500,0.314692,2011,Q4,2011-12-31,0.993098


In [45]:
df_growth_portfolio.isnull().sum()

gvkey                          0
permno                         0
ticker                         2
cumretx_q                      0
beme                           0
roa                            0
operating_margin              53
gross_margin                 483
revenue_growth               712
capex_intensity             1924
roa_stability                495
revenue_growth_stability    5452
year                           0
quarter                        0
jdate                          0
beme_pct_rank                  0
dtype: int64

In [46]:
df_value_portfolio.isnull().sum()

gvkey                           0
permno                          0
ticker                         27
cumretx_q                       0
beme                            0
roa                            35
operating_margin              635
gross_margin                  905
revenue_growth                910
capex_intensity             28411
roa_stability                 602
revenue_growth_stability     2969
year                            0
quarter                         0
jdate                           0
beme_pct_rank                   0
dtype: int64

In [47]:
df_g_portfolio = df_growth_portfolio.dropna(subset=[
    'operating_margin',
    'gross_margin',
    'revenue_growth',
    'capex_intensity',
    'roa_stability',
    'revenue_growth_stability'
])

In [48]:
df_v_portfolio = df_value_portfolio.dropna(subset=[
    'operating_margin',
    'gross_margin',
    'revenue_growth',
    'capex_intensity',
    'roa_stability',
    'revenue_growth_stability'
])

In [49]:
# Create quarterly summary with unique permno count and total permno count per quarter
quarterly_summary_growth = (
    df_g_portfolio
    .groupby(['year', 'quarter'])
    .agg(
        unique_permnos=('permno', 'nunique'),
        total_permnos=('permno', 'count')
    )
    .reset_index()
    .sort_values(['year', 'quarter'])
)

quarterly_summary_growth.head()

,year,quarter,unique_permnos,total_permnos
0,1975,Q1,102,102
1,1975,Q2,132,132
2,1975,Q3,102,102
3,1975,Q4,100,100
4,1976,Q1,120,120


In [50]:
# Create quarterly summary with unique permno count and total permno count per quarter
quarterly_summary_value = (
    df_v_portfolio
    .groupby(['year', 'quarter'])
    .agg(
        unique_permnos=('permno', 'nunique'),
        total_permnos=('permno', 'count')
    )
    .reset_index()
    .sort_values(['year', 'quarter'])
)

quarterly_summary_value.head()

,year,quarter,unique_permnos,total_permnos
0,1975,Q1,1161,1161
1,1975,Q2,1094,1094
2,1975,Q3,1123,1123
3,1975,Q4,1164,1164
4,1976,Q1,1023,1024


In [51]:
quarterly_summary_value.tail()

,year,quarter,unique_permnos,total_permnos
195,2023,Q4,866,866
196,2024,Q1,839,839
197,2024,Q2,943,943
198,2024,Q3,830,830
199,2024,Q4,720,720


In [52]:
df_final_growth = df_g_portfolio.sort_values(['jdate', 'permno'])
df_final_growth

,gvkey,permno,ticker,cumretx_q,beme,roa,operating_margin,gross_margin,revenue_growth,capex_intensity,roa_stability,revenue_growth_stability,year,quarter,jdate,beme_pct_rank
254321,006307,12749.0,KG,1.152542,0.340217,0.015179,0.123322,-1.433203,0.275058,0.011219,0.009632,0.242875,1975,Q1,1975-01-31,0.269444
220780,005606,27828.0,HWP,1.114345,0.270635,0.034255,0.518153,-0.55443,-0.134192,0.025455,0.005770,0.150968,1975,Q1,1975-01-31,0.197663
319066,007646,47773.0,NCH,1.139344,0.171573,0.04456,0.70208,-0.6346,-0.000048,0.003616,0.005421,0.066373,1975,Q1,1975-01-31,0.102878
61940,002269,49373.0,HRB,1.275862,0.269333,0.201333,0.399247,0.106593,2.591381,0.019595,0.096254,2.297455,1975,Q1,1975-01-31,0.196335
362778,008504,49576.0,PST,1.248175,0.304979,0.065044,0.514239,-1.145176,0.24804,0.016154,0.010699,0.217788,1975,Q1,1975-01-31,0.233265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18802,001414,93330.0,PRI,1.023647,0.24941,0.011455,0.284959,0.290445,0.018074,0.000177,0.003658,0.022809,2024,Q4,2024-12-31,0.173689
1304373,179841,93345.0,CDXS,1.548703,0.172415,-0.069632,-0.342078,0.82973,0.672251,0.011905,0.062299,0.876491,2024,Q4,2024-12-31,0.102348
1321879,183974,93356.0,SPSC,0.947571,0.123651,0.017027,0.14478,0.702329,0.044115,0.006026,0.001995,0.016474,2024,Q4,2024-12-31,0.062014
1321469,183945,93371.0,CRMD,1.002476,0.172248,0.113289,0.448254,0.966581,1.724337,0.000084,0.118385,1.954662,2024,Q4,2024-12-31,0.102163


In [53]:
df_final_value = df_v_portfolio.sort_values(['jdate', 'permno'])
df_final_value

,gvkey,permno,ticker,cumretx_q,beme,roa,operating_margin,gross_margin,revenue_growth,capex_intensity,roa_stability,revenue_growth_stability,year,quarter,jdate,beme_pct_rank
38176,001803,10559.0,DG,1.216783,1.411482,0.029561,0.209886,-1.277988,0.2992,0.009535,0.013731,0.237118,1975,Q1,1975-01-31,0.880073
69750,002436,10866.0,BWS,1.196429,1.571403,0.006945,0.160694,-1.235648,-0.061808,0.01289,0.004771,0.123990,1975,Q1,1975-01-31,0.907434
242726,006081,12503.0,HR,1.063291,2.42845,0.000188,0.14446,0.279499,-0.241011,0.012349,0.009163,0.176851,1975,Q1,1975-01-31,0.96853
251747,006260,12626.0,JWT,1.309353,1.091632,0.014868,0.093152,-2.041581,0.086993,0.008869,0.003705,0.261034,1975,Q1,1975-01-31,0.796514
285550,006928,12976.0,MZ,1.306122,1.946539,-0.001623,0.253551,-1.942385,0.291445,0.020291,0.013449,0.271046,1975,Q1,1975-01-31,0.945783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184478,145407,93302.0,CREG,0.872727,15.556524,-0.005007,-inf,NaN,NaN,0.0,0.001322,0.377444,2024,Q4,2024-12-31,0.998887
1323710,184167,93304.0,FIBK,1.058344,0.973457,0.001788,0.281116,0.610515,0.00269,0.000227,0.000201,0.035439,2024,Q4,2024-12-31,0.741361
1156230,133764,93368.0,EBMT,0.970869,1.422003,0.001632,0.232605,0.684539,-0.015247,0.00158,0.000310,0.066104,2024,Q4,2024-12-31,0.877684
1330866,185138,93426.0,VPG,0.906178,1.122973,0.001703,0.007956,0.438867,-0.040593,0.004874,0.006729,0.047183,2024,Q4,2024-12-31,0.800949


In [54]:
df_final_growth['portfolio_type'] = 'Growth'
df_final_value['portfolio_type'] = 'Value'

# Concatenate both datasets
df_combined_portfolio = pd.concat([df_final_growth, df_final_value], ignore_index=True)

In [55]:
# Save to CSV
df_combined_portfolio.to_csv('stocks_portfolio_overall.csv', index=False)

In [ ]:
df_final_growth.isnull().sum()

In [ ]:
df_final_value.isnull().sum()